In [60]:
import theano
import theano.tensor as TT
import numpy
import argparse
import cPickle
import traceback
import logging
import copy
import time
import sys
from collections import OrderedDict
from experiments.nmt import \
    RNNEncoderDecoder, \
    prototype_phrase_state, \
    parse_input
from experiments.nmt.numpy_compat import argpartition
from isles_utils import compute_mouse_movements, find_isles, is_sublist, subfinder
from experiments.nmt.online.online_utils import create_batch_from_seqs, loadSourceLanguageFromState, \
    loadTargetLanguageFromState
from groundhog.datasets.UnbufferedDataIterator import UnbufferedDataIterator
from groundhog.trainer.SGD_online import SGD
from groundhog.trainer.SGD_adadelta_online import SGD as Adadelta

logger = logging.getLogger(__name__)
state = prototype_phrase_state()
statepkl = '/media/HDD_2TB/MODELS/xerox/esen/models/xerox_289_354_state.pkl'
with open(statepkl) as src:
    state.update(cPickle.load(src))
sourceLanguage = loadSourceLanguageFromState(state)
targetLanguage = loadTargetLanguageFromState(state)
i=0
rng = numpy.random.RandomState(state['seed'])
enc_decs = []
enc_decs.append(RNNEncoderDecoder(state, rng, skip_init=True, compute_alignment=1))
enc_decs[i].build()
lm_models=[]
lm_models.append(enc_decs[i].create_lm_model())
models =['/media/HDD_2TB/MODELS/xerox/esen/models/xerox_289_354_model_bleu50.npz']
source = '/media/HDD_2TB/DATASETS/xerox/DATA/test.es'
trans = '/media/HDD_2TB/DATASETS/xerox/DATA/test.en'
num_sentences = 1
batch_iters=[]
batch_iters.append(UnbufferedDataIterator(source, trans, state,
                                          sourceLanguage.word_indx,
                                          targetLanguage.word_indx, 
                                          sourceLanguage.indx_word,
                                          targetLanguage.indx_word,
                                          num_sentences, state['seqlen'], None))
print "Model loaded"

Model loaded


In [146]:
class AdaGrad(object):
    """
    Adagrad [1] is an algorithm for gradient-based optimization 
    that adapts the learning rate to the parameters, performing
    larger updates for infrequent and smaller updates for frequent parameters.
    
    We set g_{t,i} to be the gradient of the objective function w.r.t. to the parameter θi at time step t
    gt,i=∇θJ(θi)
    Adagrad modifies the general learning rate η at each time step t for every parameter θi based
    on the past gradients that have been computed for θi:
    θ{t+1,i}=θ{t,i}−\div{η}{\sqrt{G_{t,ii}+ϵ}}*g_{t,i}

    Gt∈ℝ^{d×d}: Diagonal matrix where each diagonal element i,i is the sum of the squares of the gradients w.r.t. θi up to time step t


    """
    def __init__(self,
                 model,
                 state,
                 data):

        #####################################
        # Step 0. Constructs shared variables
        #####################################

        bs = state['bs']
        self.model = model
        self.rng = numpy.random.RandomState(state['seed'])

        self.add_noise = state['weight_noise']
        self.step = 0
        self.bs = bs
        self.state = state
        self.data = data
        self.step_timer = time.time()
        self.gdata = [theano.shared(numpy.zeros( (2,)*x.ndim,
                                                dtype=x.dtype),
                                    name=x.name) for x in model.inputs]
        self.gs = [theano.shared(numpy.zeros(p.get_value(borrow=True).shape,
                                             dtype=theano.config.floatX),
                                name=p.name)
                   for p in model.params]
        
        self.eps = 1e-4
        if 'profile' not in self.state:
            self.state['profile'] = 0

        ###################################
        # Step 1. Compile training function
        ###################################
        print 'Constructing grad function'
        loc_data = self.gdata
        lr = TT.scalar('lr')
        self.prop_names = [x[0] for x in model.properties]
        self.prop_exprs = [x[1] for x in model.properties]
        self.update_rules = [x[1] for x in model.updates]
        inputs_replacement_list = zip(model.inputs, loc_data)

        parameter_gradients = theano.clone(model.param_grads, replace = inputs_replacement_list)

            
        accumulated_squared_gradients = [theano.shared(numpy.zeros(param.shape.eval(),
                                                                   dtype=param.dtype),
                                                       name = param.name + '_acc_grad')
                                         for param in model.params]

        output_values = []
        st = time.time()
        
        def shift_zeroes(x):
            # The formula would create NaN upon the presence of zeroes
            return x + (abs(x) < 1e-8) * 1e-8
        
        
        def nan_and_inf_to_zero(x):
            x = TT.switch(TT.isnan(x), 0.0, x)
            return TT.switch(TT.isinf(x), 1e8, x)
        

        no_nan_or_inf_gradients = [nan_and_inf_to_zero(gradient) for gradient in parameter_gradients]
        accumulated_squared_gradients_update = [(acc_gradient, acc_gradient + gradient**2)
                                        for acc_gradient, gradient in
                                        zip(accumulated_squared_gradients, no_nan_or_inf_gradients)]
        
        
        weight_update_list = [(weight, weight - lr / TT.sqrt(TT.inc_subtensor(G_t[1][:],  1e-8)+TT.pow(gradient, 2))
                               * gradient) for weight, gradient, G_t 
                              in zip(model.params, no_nan_or_inf_gradients, accumulated_squared_gradients_update)]
        
        
        updates = weight_update_list + accumulated_squared_gradients_update
        print 'Compiling update function'
        self.lr = numpy.float32(state['lr'])
        print '\t > Using a learning rate of', self.lr
        self.update_fn = theano.function(
            [lr], output_values, name='update_function',
            allow_input_downcast=True,updates = updates,
            profile=self.state['profile'],
            )

        print 'took', time.time() - st

        self.old_cost = 1e20
        self.schedules = model.get_schedules()
        self.return_names = self.prop_names + \
                ['cost',
                 'time_step',
                 'whole_time']

    def __call__(self, _):
        """
        Ignored parameter: hypothesis.
        """
        batch = self.data.next()

        """
        # Perturb the data (! and the model)
        if self.add_noise:
            if isinstance(batch, dict):
                batch = self.model.perturb(**batch)
            else:
                batch = self.model.perturb(*batch)
        """
        # Load the dataset into GPU
        # Note: not the most efficient approach in general, as it involves
        # each batch is copied individually on gpu
        if isinstance(batch, dict):
            for gdata in self.gdata:
                # print batch[gdata.name]
                gdata.set_value(batch[gdata.name], borrow=True)
        else:
            for gdata, data in zip(self.gdata, batch):
                gdata.set_value(data, borrow=True)

        rvals = self.update_fn()
        if len(rvals) > 0:
            print rvals

In [147]:
#from groundhog.trainer.SGD_adagrad import AdaGrad
algos = []
algos.append(eval('AdaGrad')(lm_models[i], state, batch_iters[i]))

Constructing grad function
Compiling update function
	 > Using a learning rate of 1.0


MissingInputError: An input of the graph, used to compute Subtensor{::int64}(x, Constant{-1}), was not provided and not given a value.Use the Theano flag exception_verbosity='high',for more information on this error.

Backtrace when the variable is created:
  File "/home/lvapeab/software/anaconda/lib/python2.7/site-packages/ipykernel/kernelbase.py", line 228, in dispatch_shell
    handler(stream, idents, msg)
  File "/home/lvapeab/software/anaconda/lib/python2.7/site-packages/ipykernel/kernelbase.py", line 391, in execute_request
    user_expressions, allow_stdin)
  File "/home/lvapeab/software/anaconda/lib/python2.7/site-packages/ipykernel/ipkernel.py", line 199, in do_execute
    shell.run_cell(code, store_history=store_history, silent=silent)
  File "/home/lvapeab/software/anaconda/lib/python2.7/site-packages/IPython/core/interactiveshell.py", line 2723, in run_cell
    interactivity=interactivity, compiler=compiler, result=result)
  File "/home/lvapeab/software/anaconda/lib/python2